# Extract and Visualize Gradient Attribution

In [1]:
import pandas as pd
import numpy as np
import re

SEED = 42


from transformers import (DataCollatorWithPadding, Trainer, TrainingArguments,
                          LongformerTokenizer, LongformerForSequenceClassification,
                          LongformerConfig)

from transformers.models.longformer.modeling_longformer import create_position_ids_from_input_ids

from datasets import Dataset, DatasetDict

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

import torch
from torch.utils.data import DataLoader
assert torch.cuda.is_available(), 'GPU not found. You should fix this.'

In [2]:
from captum.attr import Saliency, LayerIntegratedGradients, IntegratedGradients
from captum.attr import visualization as viz

## Data

In [3]:
def get_datadict(score_to_predict):
    
    scores = {
        'Overall',
        'Cohesion',
        'Syntax',
        'Vocabulary',
        'Phraseology',
        'Grammar',
        'Conventions'
    }
    
    columns_to_remove = scores.symmetric_difference([score_to_predict])
    
    dd = (DatasetDict
          .load_from_disk('../data/ellipse.hf')
          .remove_columns(columns_to_remove)
          .rename_column(score_to_predict, 'label')
         )
    
    return dd

In [42]:
score_to_predict = 'Grammar'

dd = get_datadict(score_to_predict)
dd = dd.with_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

dataloader = DataLoader(ds['dev'].shuffle().select(range(10)), batch_size=1)

## Model and Tokenizer

In [5]:
model_chkpt = '../bin/checkpoint-284/'
model = LongformerForSequenceClassification.from_pretrained(model_chkpt, num_labels=1).cuda()
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')

## Helper Functions

In [33]:
def forward_func(input_embedding, attention_mask, global_attention_mask):
    return model(
        inputs_embeds=input_embedding,
        attention_mask=attention_mask,
        global_attention_mask=global_attention_mask,
    ).logits

In [7]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

In [30]:
def create_viz_record(attribution_summary=None, pred_score=None, true_score=None, tokens=None):
    return viz.VisualizationDataRecord(
        attribution_summary, # token attributions
        torch.max(torch.softmax(pred_score[0], dim=0)), # pred_prob
        round(pred_score.item(), 2), # pred_class
        true_score, # true_class
        None, # attr_class
        attribution_summary.sum(), # attr_score
        tokens, # raw_input_ids
        None # convergence score
    )

### Saliency with Interpretable Embedding layer

In [44]:
# Captum function for calculating saliency attributions
saliency = Saliency(forward_func)

# Container for VisualizationDataRecords
vizs = []

for sample in dataloader:
    
    # We will use this dict to store information necessary to visualize saliency
    viz_dict = {}
    
    # Get the actual score
    viz_dict['true_score'] = round(sample.pop('label').item(), 2)  
    
    # Push tensors to GPU
    sample = {k: v.cuda() for k, v in sample.items()}

    # We need to create the embeddings manually so we can track their gradients
    manual_embed = model.longformer.embeddings(
        input_ids=sample['input_ids'],
    )

    # Global attention is only on the first token, which is <s>
    sample['global_attention_mask'] = torch.zeros_like(sample['input_ids'])
    sample['global_attention_mask'][:, 0] = 1

    # Get the plain text version of the tokens (but leave them separated by whitespace)
    viz_dict['tokens'] = [
        t.replace('Ġ', '')
        for t in 
        tokenizer.convert_ids_to_tokens(
            sample['input_ids'][0].detach().tolist()
        )
    ]

    # Calculate the predicted score separately, because the Saliency module does not output it.
    with torch.no_grad():    
        viz_dict['pred_score'] = forward_func(
            manual_embed,
            sample['attention_mask'],
            sample['global_attention_mask']
        )[0]
    
    # Use Saliency module to track gradients across a forward pass
    attribution = saliency.attribute(
        inputs=manual_embed,
        additional_forward_args=(sample['attention_mask'],
                                 sample['global_attention_mask']),
        abs=False # Do not use the absolute value.
    )
    
    viz_dict['attribution_summary'] = summarize_attributions(attribution)

    vizs.append(create_viz_record(**viz_dict))

In [46]:
print('\033[1m', 'Visualizations', '\033[0m')
display(viz.visualize_text(vizs))
print()

 Visualizations 
